In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Install Dependencies and Import Libraries

In [ ]:
!pip install ultralytics


In [ ]:
import os
import glob
import shutil
import random
import numpy as np
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

In [ ]:
# Setting random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [ ]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Loading the Dataset 


In [ ]:
img_dir = "/kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/train/images"
label_dir = "/kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/train/labels"
test_dir = "/kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images"


In [ ]:
# Creating YOLO dataset structure
dataset_path = "/kaggle/working/mosquito-dataset"
os.makedirs(f"{dataset_path}/images/train", exist_ok=True)
os.makedirs(f"{dataset_path}/images/val", exist_ok=True)
os.makedirs(f"{dataset_path}/labels/train", exist_ok=True)
os.makedirs(f"{dataset_path}/labels/val", exist_ok=True)

In [ ]:
# Getting all images and split into train/val
img_files = glob.glob(os.path.join(img_dir, "*.jpeg"))
train_files, val_files = train_test_split(img_files, test_size=0.2, random_state=SEED)


In [ ]:
# Function to move image-label pairs
def move_files(file_list, src_img_dir, src_label_dir, dest_img_dir, dest_label_dir):
    for file in file_list:
        filename = os.path.basename(file)
        label_file = os.path.join(src_label_dir, filename.replace(".jpeg", ".txt"))
        
        shutil.copy(file, os.path.join(dest_img_dir, filename))
        if os.path.exists(label_file):
            shutil.copy(label_file, os.path.join(dest_label_dir, filename.replace(".jpeg", ".txt")))
        else:
            # Create an empty label file if none exists to avoid errors
            with open(os.path.join(dest_label_dir, filename.replace(".jpeg", ".txt")), 'w') as f:
                pass

# Moving files
move_files(train_files, img_dir, label_dir, f"{dataset_path}/images/train", f"{dataset_path}/labels/train")
move_files(val_files, img_dir, label_dir, f"{dataset_path}/images/val", f"{dataset_path}/labels/val")


In [ ]:
# Printing dataset statistics
print(f"Total images: {len(img_files)}")
print(f"Training images: {len(train_files)}")
print(f"Validation images: {len(val_files)}")

In [ ]:
# Creating Dataset YAML File
dataset_yaml = f"""
path: {dataset_path}
train: images/train
val: images/val
test: {test_dir}

nc: 6
names: ["aegypti", "albopictus", "anopheles", "culex", "culiseta", "japonicus/koreicus"]
"""

with open(f"{dataset_path}/dataset.yaml", "w") as f:
    f.write(dataset_yaml)

In [ ]:
torch.cuda.empty_cache()

## Initializing the Model

In [ ]:
# Loading a pretrained model
model = YOLO('yolov8m.pt') 

In [ ]:
import torch
torch.cuda.empty_cache()


## Training the Model

In [ ]:
# Training with optimized parameters for this task
results = model.train(
    data=f"{dataset_path}/dataset.yaml", 
    epochs=50,              
    batch=16,                  
    imgsz=512,               
    seed=SEED,
    patience=5,             
    optimizer="AdamW",       # AdamW optimizer (better generalization)
    lr0=0.0005,               # Learning rate
    weight_decay=0.0005,     # Regularization
    flipud=0.0,              # No vertical flipping (mosquito orientation matters)
    fliplr=0.5,              # Horizontal flip augmentation
    mosaic=0.8,              # Mosaic augmentation
    mixup=0.1,               # Mixup augmentation
    copy_paste=0.1,          # Copy-paste augmentation
    workers=4,               
    cache=True,              # Caching dataset for faster training
    verbose=False,           
    save=True,
    device="cuda"            
)

In [ ]:
# Validating the model to check performance
metrics = model.val()
print(f"mAP@0.5: {metrics.box.map50}, mAP@0.5:0.95: {metrics.box.map}")

## Predicting on Test Dataset 

In [ ]:
results = model.predict(
    source=test_dir, 
    save_txt=True, 
    save_conf=True,
    conf=0.1,                # Confidence threshold for predictions
    iou=0.5,                 # IoU threshold for NMS
    max_det=300,               # Maximum detections per image
    agnostic_nms=True,        # Class-agnostic NMS
    augment=True,            
    visualize=True,         
    device="cuda"   
)

In [ ]:
# Getting the directory where predictions were saved
predict_dir = model.predictor.save_dir
pred_dir = os.path.join(predict_dir, "labels")

## Preparing the Submission CSV

In [ ]:
import os
import glob
import pandas as pd
import cv2

# Defining class label mapping
class_labels = {
    "0": "aegypti",
    "1": "albopictus",
    "2": "anopheles",
    "3": "culex",
    "4": "culiseta",
    "5": "japonicus/koreicus"
}

# Function to get actual image dimensions
def get_image_size(image_path):
    img = cv2.imread(image_path)
    if img is not None:
        img_height, img_width, _ = img.shape
        return img_width, img_height
    else:
        print(f"Warning: Unable to read {image_path}")
        return None, None

# Getting all test image files with full filenames (including .jpeg extension)
test_image_files = glob.glob(os.path.join(test_dir, "*.jpeg"))
test_image_filenames = [os.path.basename(file) for file in test_image_files]
test_image_filenames.sort()  # Sort for consistent ordering

submission_data = []

for idx, img_filename in enumerate(test_image_filenames):
    img_id = img_filename.replace(".jpeg", "")  
    pred_file_path = os.path.join(pred_dir, f"{img_id}.txt")
    img_path = os.path.join(test_dir, img_filename)
    
    img_width, img_height = get_image_size(img_path)  # Getting actual image size
    
    if img_width is None or img_height is None:
        print(f"Skipping {img_filename} due to missing image dimensions.")
        continue
    
    if os.path.exists(pred_file_path) and os.path.getsize(pred_file_path) > 0:
        detections = []
        with open(pred_file_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 6:  # Class, coordinates, and confidence
                    class_id = parts[0]
                    x_center, y_center, width, height = map(float, parts[1:5])
                    confidence = float(parts[5])

                    # Converting normalized coordinates to pixel values
                    x_center = float(parts[1])  
                    y_center = float(parts[2])
                    width = float(parts[3])
                    height = float(parts[4])


                    detections.append({
                        "class": class_id,
                        "confidence": confidence,
                        "x_center": x_center,
                        "y_center": y_center,
                        "width": width,
                        "height": height
                    })
        
        if detections:
            # Sort by confidence (highest first)
            detections.sort(key=lambda x: x["confidence"], reverse=True)
            best_detection = detections[0]
            
            submission_data.append({
                "id": idx,
                "ImageID": img_filename,
                "LabelName": class_labels.get(best_detection["class"], "albopictus"),
                "Conf": best_detection["confidence"],
                "xcenter": best_detection["x_center"],
                "ycenter": best_detection["y_center"],
                "bbx_width": best_detection["width"],
                "bbx_height": best_detection["height"]
            })
        else:
            
            submission_data.append({
                "id": idx,
                "ImageID": img_filename,
                "LabelName": "albopictus",
                "Conf": 0.5,  # Default confidence
                "xcenter": 0.5,
                "ycenter": 0.5,
                "bbx_width": 0.2,
                "bbx_height": 0.2
            })
    else:
        
        submission_data.append({
            "id": idx,
            "ImageID": img_filename,
            "LabelName": "albopictus",
            "Conf": 0.5,
            "xcenter": 0.5,
            "ycenter": 0.5,
            "bbx_width": 0.2,
            "bbx_height": 0.2
        })

# Creating submission dataframe
submission_df = pd.DataFrame(submission_data)

# Loading sample submission as template
sample_submission = pd.read_csv("/kaggle/input/dlp-object-detection-week-10/sample_submission.csv")

# Mapping predictions to sample submission format
prediction_map = {row["ImageID"]: row for _, row in submission_df.iterrows()}

final_submission = sample_submission.copy()

for i, row in final_submission.iterrows():
    img_id = row['ImageID']
    if img_id in prediction_map:
        final_submission.at[i, 'LabelName'] = prediction_map[img_id]['LabelName']
        final_submission.at[i, 'Conf'] = prediction_map[img_id]['Conf']
        final_submission.at[i, 'xcenter'] = prediction_map[img_id]['xcenter']
        final_submission.at[i, 'ycenter'] = prediction_map[img_id]['ycenter']
        final_submission.at[i, 'bbx_width'] = prediction_map[img_id]['bbx_width']
        final_submission.at[i, 'bbx_height'] = prediction_map[img_id]['bbx_height']


expected_columns = ['id', 'ImageID', 'LabelName', 'Conf', 'xcenter', 'ycenter', 'bbx_width', 'bbx_height']
final_submission = final_submission[expected_columns]

# Saving the final submission
final_submission.to_csv("/kaggle/working/21F3000728.csv", index=False)
print(f"Final submission saved with {len(final_submission)} entries.")
print("Sample of submission format:")
print(final_submission.head())
